# Fitting a matrix transformation

For each point, we have (up to measurement error),

$$
\begin{pmatrix}
  x_1&x_2&1&0&0&0\\
  0&0&0&x_1&x_2&1
\end{pmatrix}\begin{pmatrix}
  F_{11}\\
  F_{12}\\
  c_1\\
  F_{21}\\
  F_{22}\\
  c_2
\end{pmatrix}=\begin{pmatrix}
  r_1\\
  r_2
\end{pmatrix}
$$

vcat all the points gives the standard matrix form of least squares problem. The
adapter between the two problems is implemented as `fit_F` in [`lsq.jl`](lsq.jl)

1. See [$F_a$](#$F_a$) below

2. See [$F_b$](#$F_b$) and the [difference](#Difference) below.


In [2]:
include("lsq.jl")

fit_F (generic function with 1 method)

In [1]:
data = readdlm("beads.txt", ' ');
x = sub(data, (:, 1:2));
r = sub(data, (:, 3:4));

#### $F_a$

In [4]:
F_a, c_a = fit_F(x, r)

(
2x2 Array{Float64,2}:
  1.47806   0.979544
 -0.950321  1.9868  ,

[0.010102783872577811,-0.04972384480214973])

#### $F_b$

In [5]:
F_b, c_b = fit_F(r, x)

(
2x2 Array{Float64,2}:
 0.502728  -0.249489
 0.239929   0.376422,

[-0.0051828939740244055,0.027037170518102102])

#### Difference

In [11]:
F_a - inv(F_b)

2x2 Array{Float64,2}:
 -0.0330822  -0.0220283
  0.012871   -0.0313974